In [23]:
import pandas as pd
# pd.set_option("display.float_format", "{:.3f}".format)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import Bunch
from sklearn.metrics import (
    balanced_accuracy_score,
    roc_auc_score,
    accuracy_score,
    recall_score,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    RocCurveDisplay)
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import shap
import seaborn as sns
from IPython.display import display, Markdown, Latex
import random
random.seed(42)
import os
 
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.set_seed(seed)
    random.seed

In [40]:
dff = pd.read_csv('/Users/kayadetunji/Documents/AWI_GEN_PROJECT/cvd_data_awigenv1.csv').drop(columns = 
       ['Unnamed: 0', 'study_id', 'chf_meds_qc','angina_meds_qc', 'pain_action_stop_or_slow_qc', 'pain_action_rest_qc',
       'pain_action_medicine_qc', 'pain_action_carry_on_qc',
       'relief_standstill_qc', 'relief_standstill_qc.1', 'pain_location_1_qc',
       'pain_location_2_qc', 'pain_location_3_qc', 'pain_location_4_qc',
       'pain_location_5_qc', 'pain_location_6_qc', 'pain_location_7_qc',
       'pain_location_8_qc', 'pain_location_9_qc', 'pain_location_10_qc',
       'pain_location_11_qc', 'pain_location_12_qc', 'pain_location_13_qc',
       'pain_location_14_qc', 'pain_location_15_qc', 'pain_location_16_qc',
       'pain_location_17_qc', 'pain_location_18_qc','angina_treat_now_qc', 'angina_traditional_qc', 'pain_qc', 'pain2_qc'])

In [29]:
dff = pd.read_csv('/Users/kayadetunji/Downloads/HAALSI_work/project2_data_21_04_23.csv')

#### Slice out urban sites: Nairobi and Soweto

In [41]:
site_id = 3 # Nairobi
Nairobi_dff = dff[(dff['site'] == site_id) & (dff['age'] <= 60) & (dff['age'] >= 40)].copy()

site_id = 6 # Soweto
Soweto_dff = dff[(dff['site'] == site_id) & (dff['age'] <= 60) & (dff['age'] >= 40)].copy()

In [11]:
# Simple Function to calcuate CVD
def calculate_CVD(a,b,c,d,e):
    col=0
    if (int(a)==1 or int(b)==1 or int(c)==1 or int(d)==1 or int(e)==1):
        col=1
    elif (int(a)==0 and int(b)==0 and int(c)==0 and int(d)==0 and int(e)==0):
        col = 0
    elif (int(a)==pd.isna and int(b)==pd.isna and int(c)==pd.isna and int(d)==pd.isna and int(e)==pd.isna):
        col = 0
    return col

In [7]:
com=[]
for i in range(dff.shape[0]):
    com.append(calculate_CVD(dff.stroke_qc.iloc[i], dff.transient_ischemic_attack_qc.iloc[i], dff.heartattack_qc.iloc[i], dff.congestive_heart_failure_qc.iloc[i], dff.angina_qc.iloc[i]))
y1=pd.DataFrame(com, columns=['CVD'])

In [12]:
y1.value_counts(normalize=True)*100

CVD
0      95.726056
1       4.273944
dtype: float64

In [54]:
Soweto_dff.hiv_final_status_c.value_counts(normalize=True)*100

0    58.460779
2    25.801677
1    15.737543
Name: hiv_final_status_c, dtype: float64

In [43]:
dff.loc[dff['site']==6]

,site,age,sex,number_of_children_c,partnership_status_c_qc,highest_level_of_education_qc,occupation_qc,ses_site_quintile_c_qc,smoking_status_c_qc,alcohol_use_status_c_qc,...,heartattack_traditional_qc,congestive_heart_failure_qc,chf_treatment_yn_qc,chf_treat_now_qc,chf_traditional_qc,cadiovascular_current,ckd_c_qc,htn_jnc7_qc,diabetes_status_c_qc,bmi_c_qc
352,6,40,0,NaN,2,2,1,3,0,-999,...,-999,-999,-999,-999,-999,NaN,-999,0,0,22.26
369,6,43,0,NaN,-999,-999,1,-999,0,-999,...,-999,-999,-999,-999,-999,NaN,-999,0,0,29.85
377,6,41,0,NaN,-999,1,1,2,0,-999,...,-999,-999,-999,-999,-999,NaN,-999,0,0,20.98
379,6,59,0,NaN,-999,-999,4,-999,1,-999,...,-999,-999,-999,-999,-999,NaN,-999,1,0,26.32
397,6,53,0,NaN,2,1,4,3,0,-999,...,-999,-999,-999,-999,-999,NaN,-999,0,0,41.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11981,6,48,0,NaN,-999,-999,1,-999,0,-999,...,-999,-999,-999,-999,-999,NaN,-999,0,1,35.41
11985,6,57,0,NaN,-999,-999,4,-999,2,-999,...,-999,-999,-999,-999,-999,NaN,-999,0,0,56.46
12000,6,41,0,NaN,1,2,1,1,0,-999,...,-999,-999,-999,-999,-999,NaN,-999,0,0,27.10
12025,6,44,1,5.0,1,2,0,5,2,1,...,-999,-999,-999,-999,-999,0.0,0,0,0,39.20


In [47]:
print('CVD prevalence in Nairobi:\t', Nairobi_dff.cadiovascular_current.value_counts(normalize=True)[1]*100)
print('CVD prevalence in Soweto: \t', Soweto_dff.cadiovascular_current.value_counts(normalize=True)[1]*100)

CVD prevalence in Nairobi:	 4.27394438722966
CVD prevalence in Soweto: 	 3.51219512195122


In [48]:
print('CKD prevalence in Nairobi:\t', Nairobi_dff.ckd_c_qc.value_counts(normalize=True)[1]*100)
print('CKD prevalence in Soweto: \t', Soweto_dff.ckd_c_qc.value_counts(normalize=True)[1]*100)

CKD prevalence in Nairobi:	 11.946446961894953
CKD prevalence in Soweto: 	 6.364084854464726


In [49]:
print('HT prevalence in Nairobi:\t', Nairobi_dff.htn_jnc7_qc.value_counts(normalize=True)[1]*100)
print('HT prevalence in Soweto: \t', Soweto_dff.htn_jnc7_qc.value_counts(normalize=True)[1]*100)

HT prevalence in Nairobi:	 26.416065911431513
HT prevalence in Soweto: 	 53.23137641835225


In [50]:
print('DM prevalence in Nairobi:\t', Nairobi_dff.diabetes_status_c_qc.value_counts(normalize=True)[1]*100)
print('DM prevalence in Soweto: \t', Soweto_dff.diabetes_status_c_qc.value_counts(normalize=True)[1]*100)

DM prevalence in Nairobi:	 6.797116374871266
DM prevalence in Soweto: 	 9.422792303897385


In [55]:
print('HIV prevalence in Nairobi:\t', Nairobi_dff.hiv_final_status_c.value_counts(normalize=True)[1]*100)
print('HIV prevalence in Soweto: \t', Soweto_dff.hiv_final_status_c.value_counts(normalize=True)[1]*100)

HIV prevalence in Nairobi:	 12.306900102986612
HIV prevalence in Soweto: 	 15.73754316724223


In [44]:
Soweto_dff.cadiovascular_current.value_counts(normalize=True)[1]*100

3.51219512195122

In [56]:
Soweto_dff

,site,age,sex,number_of_children_c,partnership_status_c_qc,highest_level_of_education_qc,occupation_qc,ses_site_quintile_c_qc,smoking_status_c_qc,alcohol_use_status_c_qc,...,heartattack_traditional_qc,congestive_heart_failure_qc,chf_treatment_yn_qc,chf_treat_now_qc,chf_traditional_qc,cadiovascular_current,ckd_c_qc,htn_jnc7_qc,diabetes_status_c_qc,bmi_c_qc
352,6,40,0,NaN,2,2,1,3,0,-999,...,-999,-999,-999,-999,-999,NaN,-999,0,0,22.26
369,6,43,0,NaN,-999,-999,1,-999,0,-999,...,-999,-999,-999,-999,-999,NaN,-999,0,0,29.85
377,6,41,0,NaN,-999,1,1,2,0,-999,...,-999,-999,-999,-999,-999,NaN,-999,0,0,20.98
379,6,59,0,NaN,-999,-999,4,-999,1,-999,...,-999,-999,-999,-999,-999,NaN,-999,1,0,26.32
397,6,53,0,NaN,2,1,4,3,0,-999,...,-999,-999,-999,-999,-999,NaN,-999,0,0,41.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11981,6,48,0,NaN,-999,-999,1,-999,0,-999,...,-999,-999,-999,-999,-999,NaN,-999,0,1,35.41
11985,6,57,0,NaN,-999,-999,4,-999,2,-999,...,-999,-999,-999,-999,-999,NaN,-999,0,0,56.46
12000,6,41,0,NaN,1,2,1,1,0,-999,...,-999,-999,-999,-999,-999,NaN,-999,0,0,27.10
12025,6,44,1,5.0,1,2,0,5,2,1,...,-999,-999,-999,-999,-999,0.0,0,0,0,39.20
